In [1]:
from pyspark.context import SparkContext
sc=SparkContext.getOrCreate()

heartRdd=sc.textFile("/FileStore/tables/data.csv")
heartRdd.cache()
heartRdd.collect()

In [2]:
#Remove the first line (contains headers)
heartRdd = heartRdd.filter(lambda x: "cp" not in x)
heartRdd.count()
heartRdd.collect()

In [3]:
import math
from pyspark.ml.linalg import Vectors, VectorUDT
#from pyspark.ml.linalg import Vectors

avgChol =sc.broadcast(250)
avgFbs = sc.broadcast(0.06)
avgRrestecg =sc.broadcast(0.21)
avgThalach=sc.broadcast(139.1)
avgExang=sc.broadcast(0.3)
avgSlop=sc.broadcast(1.89)
avgCa=sc.broadcast(0)
avgThal=sc.broadcast(5.6)
avgTres=sc.broadcast(132.5)

def transformToVector( inputStr) :

    attList=inputStr.split(",")    
    
    cholValue = attList[4]
    if cholValue == "?":
        cholValue=avgChol.value
    fbsValue = attList[5]
    if fbsValue == "?":
        fbsValue=avgFbs.value
    restecgValue = attList[6]
    if restecgValue == "?":
        restecgValue=avgRrestecg.value
    thalachValue = attList[7]
    if thalachValue == "?":
        thalachValue=avgThalach.value
    exangValue = attList[8]
    if exangValue == "?":
        exangValue=avgExang.value
    slopValue = attList[10]
    if slopValue == "?":
        slopValue=avgSlop.value
    caValue = attList[11]
    if caValue == "?":
        caValue=avgCa.value
    thalValue = attList[11]
    if thalValue == "?":
        thalValue=avgThal.value
    tresetValue= attList[3]
    if tresetValue =="?":
       tresetValue=avgTres.value
        
    
    #Filter out columns not wanted at this stage
    values= Vectors.dense([float(attList[13]), \
                     float(attList[2]),\
                     tresetValue,\
                     cholValue,  \
                     fbsValue,  \
                     restecgValue,  \
                     thalachValue,  \
                     exangValue,  \
                     slopValue,  \
                     caValue, \
                     thalValue \
                    ])

  
    return values





In [4]:
heartVectors = heartRdd.map(transformToVector)
heartVectors.collect()

In [5]:
def transformToLabeledPoint(inStr) :
    labelPoint = (float(inStr[0]), Vectors.dense([inStr[1],inStr[2],inStr[4]]))
    return labelPoint
    
heartLp = heartVectors.map(transformToLabeledPoint)
heartDF = sqlContext.createDataFrame(heartLp,["label", "features"])
heartDF.select("label","features").show(10)



In [6]:
(trainingData, testData) = autoDF.randomSplit([0.9, 0.1])
trainingData.count()



In [7]:
testData.count()


In [8]:
#Setup pipeline
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml import Pipeline
nbClassifier=NaiveBayes()
pipeline = Pipeline(stages=[nbClassifier])
model = pipeline.fit(trainingData)

In [9]:
prediction=model.transform(testData)


In [10]:
prediction.groupBy("label","prediction").count().show()

In [11]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
dtClassifer = DecisionTreeClassifier()
dtModel = dtClassifer.fit(trainingData)


In [12]:
prediction1=dtModel.transform(testData)

In [13]:
prediction1.groupBy("label","prediction").count().show()